# Preliminaries

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
library(patchwork)

In [ ]:
ws_namespace <- Sys.getenv("WORKSPACE_NAMESPACE")
ws_name <- Sys.getenv("WORKSPACE_NAME")
ws_bucket <- Sys.getenv("WORKSPACE_BUCKET")

In [ ]:
theme_set(theme_bw())

## Read in phenotypic and metabolomic data

* Phenotype data come from integrating inputs from dbGaP (/PIC-SURE), MESA investigators, and metadata from metabolomic preprocessing.
* Genotype data come from dbGaP (Freeze 9b TOPMed-wide genotype VCF files).

In [ ]:
analysis_df <- read_csv("analysis/phenos_and_genos.csv", col_types=cols())
names(analysis_df)

analysis_df_all <- read_csv("analysis/phenos_and_genos_all.csv", col_types=cols())

In [ ]:
snp_info_df <- read_csv("genotypes/snp_info.csv", col_types=cols())

exposures <- c("pa", "pa_bin")

# Explore physical activity main effects and covariate adjustments

In [ ]:
fit_main_effect_model <- function(y, e, covars, df) {
  form_str <- paste0(y, " ~ ", e, " + ", paste(covars, collapse=" + "))
  lm_fit <- lm(as.formula(form_str), data=df) 
  lm_fit %>%
      broom::tidy() %>%
      filter(term == e)
}

basic_covars <- c("site", "gender_f0m1", "age")
covar_sets <- list(
    basic = basic_covars,
    add_ses = c(basic_covars, "ses_score", "income_cat"),
    add_ses_HL = c(basic_covars, "ses_score", "income_cat", 
                              "drinks_per_week", "smoking", "ahei_score", "dash_score"),
    add_ses_HL_gPC = c(basic_covars, "ses_score", "income_cat", 
                              "drinks_per_week", "smoking", "ahei_score", "dash_score",
                       paste0("gPC", 1:5)),
    add_ses_HL_race = c(basic_covars, "ses_score", "income_cat", 
                              "drinks_per_week", "smoking", "ahei_score", "dash_score",
                   "race")
)

pa_fields <- c("pa", "pa_bin", "mod_pa", "vig_pa", "mvpa")

main_effect_sensitivity_res_df <- expand_grid(
  e = c(pa_fields, "rs295849"),
  covar_set = names(covar_sets)
) %>%
  rowwise() %>%
  mutate(lm_fit = list(fit_main_effect_model("hdl_log", e, covar_sets[[covar_set]], analysis_df))) %>%
  unnest(lm_fit)

analysis_df_all$pa_log <- log(analysis_df_all$pa + 1)
main_effect_sensitivity_res_df_all <- expand_grid(
  e = c(pa_fields, "pa_log", "rs295849"),
  covar_set = names(covar_sets)
) %>%
  rowwise() %>%
  mutate(lm_fit = list(fit_main_effect_model("hdl_log", e, covar_sets[[covar_set]], analysis_df_all))) %>%
  unnest(lm_fit)

In [ ]:
options(repr.plot.width=16, repr.plot.height=5)

main_effect_sensitivity_res_df_all %>%
  filter(e %in% c("pa", "rs295849")) %>%
  mutate(l95 = estimate - 1.96 * std.error,
         u95 = estimate + 1.96 * std.error,
         covar_set = factor(covar_set, levels=names(covar_sets))) %>%
  ggplot(aes(x=covar_set, y=estimate)) +
  geom_point() +
  geom_errorbar(aes(ymin=l95, ymax=u95), width=0.2) +
  geom_hline(yintercept=0, color="gray") +
  facet_wrap(vars(e), scale="free_y", nrow=1) +
  labs(x="Covariate set", y="PA or SNP main effect estimate (95% CI)",
       title="Main effects in the full MESA dataset")

main_effect_sensitivity_res_df %>%
  filter(e %in% c("pa", "rs295849")) %>%
  mutate(l95 = estimate - 1.96 * std.error,
         u95 = estimate + 1.96 * std.error,
         covar_set = factor(covar_set, levels=names(covar_sets))) %>%
  ggplot(aes(x=covar_set, y=estimate)) +
  geom_point() +
  geom_errorbar(aes(ymin=l95, ymax=u95), width=0.2) +
  geom_hline(yintercept=0, color="gray") +
  facet_wrap(vars(e), scale="free_y", nrow=1) +
  labs(x="Covariate set", y="PA or SNP main effect estimate (95% CI)",
       title="Main effects in the MESA subset with LC/MS metabolomics")

It appears that both gPCs and race variables have some effect on PA effect estimates. Given this, and the multi-population nature of this dataset, we will include 5 gPCs in subsequent models.

In [ ]:
covars <- covar_sets$add_ses_HL_gPC

In [ ]:
pa_types <- c("pa", "pa_log", "pa_bin", "mvpa", "mod_pa", "vig_pa")
pa_types_clean <- c("Intentional PA", "log(Intentional PA)", "Binarized intentional PA",
                    "Moderate + vigorous PA", "Moderate PA", "Vigorous PA")

main_effect_sensitivity_res_df_all %>%
  filter(e != "rs295849",
         covar_set == "add_ses_HL_gPC") %>%
  mutate(e = factor(e, levels = pa_types, labels = pa_types_clean)) %>%
  ggplot(aes(x=e, y=statistic)) +
  geom_bar(stat="identity", width=0.5) +
  geom_hline(yintercept=0, color="gray") +
  labs(x="Covariate set", y="Z-statistic",
       title="Significance of main effects for alternative PA variables")

It also appears that vigorous PA has a substantially stronger association with HDL-C than the "intentional PA" variable used in the CHARGE Phase I meta-analysis.

# Test for the primary interactions

Can we reproduce in MESA the interactions found in the original CHARGE GLI meta-analyses?

## Previously reported GxEs

SNP-exposure-outcome triplets came from two CHARGE GLI publications:
* Kilpelainen et al. 2019, *Nat. Comm.* (https://doi.org/10.1038/s41467-018-08008-w) -- physical activity and lipids
* Bentley et al. 2019, *Nat. Genet.* (https://doi.org/10.1038/s41588-019-0378-y) -- smoking and lipids

For both of these, the exposure(s) were coded as binary variables. HDL-C was log-transformed prior to analysis.

Details on the genetic variants:

In [ ]:
head(snp_info_df)

Details on the previously reported GxE effects:

In [ ]:
gli_info_df <- tribble(
    ~SNP, ~exposure, ~outcome, ~effect_allele, ~EAF, ~beta_int, ~se_int,
    "rs2862183", "pa", "hdl_log", "T", "0.22", "-0.014", "0.003",
    "rs295849", "pa", "hdl_log", "T", "0.38", "0.009", "0.002",
    "rs141588480", "pa", "hdl_log", "Ins", "0.95", "-0.054", "0.010") %>%
    mutate(across(c(EAF, beta_int, se_int), as.numeric)) %>%
    filter(SNP != "rs141588480")

gli_info_df

## Replication of the primary GxEs in MESA

In [ ]:
test_gxe <- function(y, snp, e, covars, df) {
    form_str <- paste0(y, " ~ ", e, " * ", snp)
    if (!identical(covars, "")) form_str <- paste0(form_str, " + ", paste(covars, collapse=" + "))
    sumstats <- lm(as.formula(form_str), data=df) %>%
        broom::tidy() %>%
        filter(term %in% c(e, snp, paste0(e, ":", snp))) %>%
        mutate(EAF_topmed = sum(df[[snp]]) / (2 * nrow(df)))
    sumstats
}

gli_res_df <- gli_info_df %>%
    rowwise() %>%
    mutate(mod = list(test_gxe(outcome, SNP, exposure, 
                               covars, analysis_df))) %>%
    unnest(mod)

gli_res_df_all <- gli_info_df %>%
    rowwise() %>%
    mutate(mod = list(test_gxe(outcome, SNP, exposure, 
                               covars, analysis_df_all))) %>%
    unnest(mod)

gli_res_df_paPCadj <- gli_info_df %>%
    rowwise() %>%
    mutate(mod = list(test_gxe(outcome, SNP, exposure, 
                               c(covars, paste0("pa*gPC", 1:5)), analysis_df))) %>%
    unnest(mod)

In [ ]:
pa_subtype_gxe_res_df_all <- expand_grid(
  y = c("hdl_log"),
  e = c("pa", "mod_pa", "vig_pa", "mvpa")
) %>%
  rowwise() %>%
  mutate(lm_res = list(test_gxe(y, "rs295849", e, covars, analysis_df_all))) %>%
  unnest(lm_res)
pa_subtype_gxe_res_df_female <- expand_grid(
  y = c("hdl_log"),
  e = c("pa", "mod_pa", "vig_pa", "mvpa")
) %>%
  rowwise() %>%
  mutate(lm_res = list(test_gxe(y, "rs295849", e, covars, analysis_df_all %>% filter(gender_f0m1 == 0)))) %>%
  unnest(lm_res)
pa_subtype_gxe_res_df <- bind_rows(list(
  all = pa_subtype_gxe_res_df_all,
  female = pa_subtype_gxe_res_df_female
), .id="subgroup") %>%
  filter(grepl(":rs295849", term))

pa_subtype_gxe_res_df %>%
  mutate(l95 = estimate - 1.96 * std.error,
         u95 = estimate + 1.96 * std.error) %>%
  ggplot(aes(x=e, y=estimate, color=subgroup)) +
  geom_point(position=position_dodge(width=0.2)) +
  geom_errorbar(aes(ymin=l95, ymax=u95), width=0.2, position=position_dodge(width=0.2)) +
  geom_hline(yintercept=0, color="gray") +
  facet_wrap(~y, nrow=1, scale="free") +
  labs(x="", y="Interaction effect estimate")

pa_subtype_gxe_res_df %>%
  arrange(p.value)

### Regression results from MESA only

In [ ]:
gli_res_df %>%
    dplyr::select(SNP, EAF_topmed, exposure, outcome, term, estimate, std.error, p.value) %>%
    mutate(across(c("estimate", "std.error", "p.value", "EAF_topmed"), round, 3))

In [ ]:
gli_res_df %>%
    dplyr::select(SNP, EAF_topmed, exposure, outcome, term, estimate, std.error, p.value) %>%
    group_by(SNP, EAF_topmed, exposure) %>%
    summarise(beta_main = estimate[grepl("^rs", term)],
              p_main = p.value[grepl("^rs", term)],
              beta_1df = estimate[grepl(":", term)],
              p_1df = p.value[grepl(":", term)],
              .groups="drop")

African-American subset only:

In [ ]:
test_interaction_race_specific <- function(y, snp, e, covars, r) {
    form_str <- paste0(y, " ~ ", e, " * ", snp)
    if (!identical(covars, "")) form_str <- paste0(form_str, " + ", paste(covars, collapse=" + "))
    sumstats <- lm(as.formula(form_str), data=filter(analysis_df, race == r)) %>%
        broom::tidy() %>%
        filter(term %in% c(e, snp, paste0(e, ":", snp))) %>%
        mutate(EAF_topmed = sum(analysis_df[[snp]]) / (2 * nrow(analysis_df)))
    sumstats
}

gli_res_df_aa <- gli_info_df %>%
    rowwise() %>%
    mutate(mod = list(test_interaction_race_specific(outcome, SNP, exposure, covars, "african-american"))) %>%
    unnest(mod)

gli_res_df_aa %>%
    filter(grepl(":", term)) %>%
    dplyr::select(SNP, EAF_topmed, exposure, outcome, term, estimate, std.error, p.value) %>%
    mutate(across(c("estimate", "std.error", "p.value", "EAF_topmed"), round, 3))

### MESA results compared to Phase I meta-analysis findings

In [ ]:
bind_rows(list(
    publication = gli_res_df %>% 
        filter(grepl(":", term)) %>%
        dplyr::select(SNP, exposure, estimate=beta_int, se=se_int),
    topmed = gli_res_df %>%
        filter(grepl(":", term)) %>%
        dplyr::select(SNP, exposure, estimate, se=std.error)
), .id="source") %>%
    mutate(l95 = estimate - 1.96 * se,
           u95 = estimate + 1.96 * se,
           xlab = paste0(SNP, " x ", exposure)) %>%
    ggplot(aes(x=xlab, y=estimate, color=source)) +
    geom_point(position=position_dodge(width=0.2)) +
    geom_errorbar(aes(ymin=l95, ymax=u95), width=0.1,
                  position=position_dodge(width=0.2)) +
    geom_hline(yintercept=0, color="gray") +
    labs(x="GxE interactions from literature",
         y="Interaction effect estimate") +
    theme(axis.text.x=element_text(angle=30, hjust=0.7)) +
    coord_cartesian(ylim=c(-0.2, 0.2))

bind_rows(list(
    publication = gli_res_df %>% 
        filter(grepl(":", term)) %>%
        dplyr::select(SNP, exposure, estimate=beta_int, se=se_int),
    topmed = gli_res_df_aa %>%
        filter(grepl(":", term)) %>%
        dplyr::select(SNP, exposure, estimate, se=std.error)
), .id="source") %>%
    mutate(l95 = estimate - 1.96 * se,
           u95 = estimate + 1.96 * se,
           xlab = paste0(SNP, " x ", exposure)) %>%
    ggplot(aes(x=xlab, y=estimate, color=source)) +
    geom_point(position=position_dodge(width=0.2)) +
    geom_errorbar(aes(ymin=l95, ymax=u95), width=0.1,
                  position=position_dodge(width=0.2)) +
    geom_hline(yintercept=0, color="gray") +
    labs(x="GxE interactions from literature",
         y="Interaction effect estimate",
         title="African-American only") +
    theme(axis.text.x=element_text(angle=30, hjust=0.7)) +
    coord_cartesian(ylim=c(-0.2, 0.2))